# Quadratically Constrained Quadratic Programs
The latest version of this notebook is available on https://github.com/Qiskit/qiskit-iqx-tutorials.

- Introduce Quadratic Programs
    - Mathematical model (incl. constraints) + variable types<br>
    https://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program
    - How to build it with docplex (brief example)
    - How to build the same with the QuadraticProgram
- Introduce Converters <br>(except "to operator" and "to neg. val. oracle", those should be introduced with the algorithms)
    - Integer to Binary Converter
    - Penalize Equality Constraints
    - ...
- Example:
    - build a model in docplex with integer variables and constraints, cast to QUBO. 
    - introduce convenience converter "to_qubo"...